# Anomaly Detection using Statistical Learning
## Spring 2022

**Presented by**: Jiachen Hu, Jiayi Wang, Marcus Loke, Qinyun Luo, Rohit Kundurthi, Xin Li, Yafei Wen, Yinjie Dai, Yunzi Ma, Zheyu Xu

In [1]:
import numpy as np
import sesd
ts = np.random.random(100)
# Introduce artificial anomalies
ts[14] = 9
ts[83] = 10
outliers_indices = sesd.seasonal_esd(ts, periodicity=20, hybrid=True, max_anomalies=2)

In [3]:
for idx in outliers_indices:
    print(f'Anomaly index: {idx}, anomaly value: {ts[idx]}')

Anomaly index: 83, anomaly value: 10.0
Anomaly index: 14, anomaly value: 9.0
